# Disclaimer & Copyright

Copyright 2024 Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Gemini - Summarize a PDF file
* This notebook explains how to use Gemini to understand pdf files in multimodality features of Gemini.
* Refer to the link for more information about the Gemini
 * ***https://cloud.google.com/vertex-ai/generative-ai/docs/learn/overview***

# Configuration
## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest

In [ ]:
%pip install --upgrade --quiet google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 11.4 MB/s eta 0:00:00


In [ ]:
from IPython.display import display, Markdown

## Authentication to access to the GCP & Google drive

* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [ ]:
#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

* Mount to the google drive to access the .ipynb files in the repository.



In [ ]:
# To access contents in Google drive

if "google.colab" in sys.modules:
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Execute the example
## Set the environment on GCP Project
* Configure project information
  * Model name : LLM model name : https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models
  * Project Id : prodect id in GCP
  * Region : region name in GCP

In [ ]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [ ]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)

# Access to the generative model.
model = GenerativeModel(MODEL_NAME)

Encoding function for multimodality

In [ ]:
import base64

def get_encoded_content(location_type, location, mime_type ):
  """
  Get the encoded content object.

  location_type :
    The type of the location. ( local or GCS )
  location :
    The file location of the content.
  mime_type :
    The mime type of the content.

  Returns:
    The encoded content object.

  """

  content_obj = None

  if location_type == "local":
    with open(location, 'rb') as f:
      raw_obj = base64.b64encode(f.read()).decode('utf-8')
      content_obj = Part.from_data(data=base64.b64decode(raw_obj), mime_type=mime_type)

  elif location_type == "GCS":
        content_obj = Part.from_uri(location, mime_type=mime_type)
  else:
    raise ValueError("Invalid location type.")

  return content_obj

### Function to get the response

In [ ]:
def generate(content_obj, query:str):
    """
    Generate a response from the model.

    content_obj :
      encoded object being analyzed in the process
    query :
      query to be sent to the model

    Returns:
      The generated response.

    """

    # Set model parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
    }

    # Configure satey setting : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # Refer to the link to remove : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }

    responses = model.generate_content(
        [content_obj, query],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=False,
    )

    return responses.text

## Run example

In [ ]:
from time import perf_counter

t1_start = perf_counter()

# When using local storage for the file location.
location_type = "local"
mime_type = "application/pdf"

repository_root = "/content/drive/MyDrive/projects/google_gen_ai_sample/"
file_path = "contents/pdfs/the-state-of-ai-in-early-2024-final.pdf"
location = repository_root + file_path

content_obj = get_encoded_content(location_type, location, mime_type )

prompt = """
You are a helpful assistant that summarizes a pdf file.
Follow the instructions below to generate a response.

1. Summarize the report by each chapter.
2. The summary of each chapter should have an analysis of the graph.
3. Explain information in the graph as detail as possible.
4. Finally summarize the whole report in 10 lines.

"""

outcome = generate(content_obj, prompt)

t1_end  = perf_counter()
print(f"Time : {t1_end - t1_start} seconds\n\n")

display(Markdown(outcome))


Time : 17.17544566300012 seconds




The report "The state of AI in early 2024: Gen AI adoption spikes and starts to generate value" explores the state of AI adoption in 2024, focusing on the rise of generative AI (gen AI). The report reveals that gen AI adoption has surged dramatically, with 65% of organizations now regularly using gen AI, nearly double the percentage from just ten months ago. This surge in adoption is driven by the recognition of the significant business value that gen AI can deliver, as organizations report both cost reductions and revenue increases from its implementation.

**Chapter 1: AI Adoption Surges**

The first chapter examines the increasing adoption of AI across industries and regions. The report highlights that AI adoption has surged in the past year, reaching 72% of organizations, after years of relatively stagnant adoption rates. This increase is driven by the growing interest in generative AI, which has expanded the scope of AI capabilities. The graph showcases the rapid increase in AI adoption from 2017 to 2024. In 2017, AI adoption was hovering around 47%.  In 2023, it climbed to 65% and in 2024, it reached 72%. This dramatic increase is indicative of the growing interest in AI and the emergence of gen AI. It also points to a shift in how organizations are using AI, with a growing emphasis on integrating AI into more core business functions. 

**Chapter 2: Survey Findings Suggest Organizations Are Using AI in More Business Functions Now than in Previous Years**

The second chapter focuses on the use of AI across business functions. The report finds that organizations are increasingly using AI in multiple business functions, with half of respondents indicating that their organizations have adopted AI in at least two business functions. This represents a significant increase from 2023, where less than a third of respondents reported using AI in multiple functions. The graph shows the number of business functions using AI in 2021, 2022, 2023, and 2024.  In 2021, 72% of respondents reported using AI in one or more business functions. That number decreased to 50% in 2023 and increased to 72% in 2024. This trend indicates that organizations are not only adopting AI but also expanding its use within their operations, seeking to leverage its benefits across a wider range of functions.

**Chapter 3: Respondents Most Often Report Generative AI Adoption in Their Marketing and Sales, Product and Service Development, and IT Functions**

This chapter examines the specific business functions where generative AI is being adopted most frequently.  The report reveals that respondents are most likely to report using gen AI in marketing and sales, product and service development, and IT functions. The graph showcases the percentage of respondents reporting regular use of gen AI in various functions. The highest percentage (34%) is found in marketing and sales, followed by product and service development (23%) and IT (17%).  This data highlights the key areas where gen AI is currently having the most impact, and it points to the significant potential of gen AI to revolutionize these business functions. 

**Chapter 4: Respondents Are Much More Likely Now than in 2023 to Say They Are Using Generative AI**

The fourth chapter explores the pervasiveness of gen AI adoption across regions, job titles, and age groups.  The report finds that respondents are significantly more likely to report using gen AI both at work and in their personal lives compared to 2023. This trend is consistent across all regions, with the most significant increases observed in Asia-Pacific and Greater China.  The graph shows the percentage of respondents who are using AI in different ways and situations, such as using it regularly at work, regularly at work and outside of work, and having tried it at least once. In 2023, the percentage of respondents using AI regularly for work was 18%. In 2024, that number climbed to 34%. This indicates a rapid adoption of AI in professional and personal life.

**Chapter 5: Investments in Gen AI and Analytical AI are Beginning to Create Value**

The fifth chapter examines the impact of gen AI and analytical AI investments on business value. The report finds that, across many industries, organizations are equally likely to invest more than 5% of their digital budgets in both gen AI and analytical AI. The report also explores the value creation achieved through the use of gen AI, with the largest share of respondents reporting cost reductions in human resources and revenue increases in supply chain and inventory management.  The graphs show the percentage of organizations investing more than 5% of their digital budgets in gen AI and analytical AI. In both categories, the percentages are fairly similar, with the technology sector showing a higher percentage for gen AI and the financial services sector showing a higher percentage for analytical AI. This indicates that organizations are increasingly investing in both forms of AI to capture their respective benefits.

**Chapter 6: Looking Ahead, Most Respondents (67%) Expect Their Organizations to Invest More in AI over the Next Three Years**

The report concludes by emphasizing the continued optimism surrounding AI investment. The majority of respondents (67%) expect their organizations to invest more in AI over the next three years, signaling a robust outlook for AI adoption and development in the years to come.

The report concludes that, while gen AI is still in its early stages, organizations are already seeing significant benefits from its adoption. However, there are also risks associated with the use of gen AI, with inaccuracy being the most frequently cited concern. Organizations are increasingly focusing on developing and implementing best practices to mitigate these risks and capture the full value of gen AI.  High performers are setting the standard for responsible AI implementation, focusing on data governance, risk mitigation, and strategic deployment.  The report emphasizes the importance of continued investment in AI and suggests that organizations should focus on building their own customized AI models to gain a competitive advantage.  This report indicates that AI adoption is accelerating and generating tangible value for organizations. However, organizations must be vigilant in managing risks and developing best practices to unlock the full potential of AI.  The report highlights the importance of collaboration, strategic planning, and a focus on value creation to ensure the successful and responsible implementation of AI across all industries.